In [1]:
import pandas as pd
import os
import dotenv
import numpy as np

dotenv.load_dotenv()
BASE_DIR = os.getcwd()

# Import excel

In [2]:
excel_path = "disposition_code_ref_table.xlsx"

df = pd.read_excel(os.path.join(BASE_DIR, excel_path))
df.drop(columns=["Konkatenasi_Layer123"], inplace=True)

# Define functions

In [3]:
def split_layer3to2(layer2_name:str, layer3_product_list:list, layer2_prefix:str="", layer2_postfix:str=""):
    for product in layer3_product_list:
        df["Layer 2"] = np.where(
                (df["Layer 2"] == layer2_name) &
                ((df["Layer 3"].str.startswith(product)) |
                 (df["Layer 3"].str.endswith(product))),
            layer2_prefix   + product + layer2_postfix,
            df["Layer 2"])
        df["Layer 3"] = np.where(
                (df["Layer 2"] == 
                layer2_prefix + product + layer2_postfix) &
                ((df["Layer 3"].str.startswith(product)) |
                 (df["Layer 3"].str.endswith(product))),
            df["Layer 3"].str.split(product).str[1],
            df["Layer 3"])
    strip_all()

def split_layer2to3(layer2_product_list:list, layer2_name:str, layer3_prefix:str="", layer3_postfix:str=""):
    for product in layer2_product_list:
        df["Layer 3"] = np.where(
                (df["Layer 2"] == product),
            layer3_prefix + product + layer3_postfix + df["Layer 3"],
            df["Layer 3"])
        df["Layer 2"] = np.where(
                (df["Layer 2"] == product),
            layer2_name,
            df["Layer 2"])
    strip_all()

def swap_layers(layer_name_1:str, layer_name_2:str, layer_name_1_contain:str="", layer_name_2_contain:str=""):
    df[layer_name_1], df[layer_name_2] = np.where(
        (df[layer_name_1].str.contains(layer_name_1_contain) &
         df[layer_name_2].str.contains(layer_name_2_contain)),
            df[layer_name_2],
            df[layer_name_1]), np.where(
        (df[layer_name_1].str.contains(layer_name_1_contain) &
         df[layer_name_2].str.contains(layer_name_2_contain)),
            df[layer_name_1],
            df[layer_name_2])

def strip_all():
    df["Layer 1"] = df["Layer 1"].str.strip()
    df["Layer 2"] = df["Layer 2"].str.strip()
    df["Layer 3"] = df["Layer 3"].str.strip()

    df.fillna(" ", inplace=True)

def group_label(layer:str, list_of_labels:list[str], result_label:str):
    strip_all()
    for label in list_of_labels:
        df.loc[df[layer].str.contains(label), layer] = result_label
    strip_all()

def remove_prefix(layer, suffix):
    df.loc[df[layer].str.startswith(suffix), layer] = df[layer].str.split(suffix).str[1].str.strip()

def remove_postfix(layer, suffix):
    df.loc[df[layer].str.endswith(suffix), layer] = df[layer].str.split(suffix).str[0].str.strip()
    # print(df[layer].str.split(suffix).str[0].str.strip())

In [4]:
def add_stage(stage):
    strstage = str(stage)
    df[["Stage "+strstage+" - Layer 1", "Stage "+strstage+" - Layer 2", "Stage "+strstage+" - Layer 3"]] = df[["Layer 1", "Layer 2", "Layer 3"]]

In [5]:
stage = 0
add_stage(stage)

Stage 1: Standardizing terms

In [6]:
# 
# 
# 
# Replace Synonyms, Typo
# print(df["Stage 0 - Layer 1"].value_counts())
df.loc[df["Layer 1"].str.contains("General Customer"), "Layer 1"] = df["Layer 1"].str.split(regex=r" (General Customer)").str[0]
strip_all()
# print(df["Layer 1"].value_counts())
df.replace({"Credit Cards": "Credit Card",
            "Kartu Kredit": "Credit Card",
            "Credit Card Suppl": "Credit Card Supplementary",
            "Kartu Debit": "Debit Card",
            "Watsapp": "Whatsapp",
            "Octo": "OCTO",
            "Transaksi  1500800": "Transaksi 1500800",
            "Pembesihan": "Pembersihan",
            r" /": "/",
            r"/ ": "/",
            r"/": "/ ",
            "Sevices": "Services",
            "Top UP": "Top Up",
            "Others": "Other",
            "E-mandate": "E-Mandate",
            "Belum Terima": "Tidak Terima",
            "Belum Menerima": "Tidak Terima",
            "Tidak Menerima": "Tidak Terima",
            "Vending Other": "Vending Machine Other",
            "CNY": "Customer Preferred Valas Chinese Yuan",
            "CP Blast": "Customer Preferred Blast",
            "CP Staff": "Customer Preferred Staff",
            "CPVALAS": "Customer Preferred Valas",
            "CPDORMANT": "Customer Preferred Dormant",
            "Done Contact Center Dok KTP Supplement": "Contact Center Dok KTP Supplement",
            "Done Telesales Dok KTP Supplement": "Telesales Dok KTP Supplement",
            # "KPR": "Kredit Pemilikan Rumah",
            # "KPM": "Kredit Pemilikan Mobil",
            "XKB ": "XKB: ",
            # "KTA": "Kredit Tanpa Agunan",
            # "PL": "Personal Loan",
            # "CDM": "CDM (Cash Deposit Machine)",
            # "CRM": "CRM (Cash Recycling Machine)",
            # "EDC": "EDC (Electronic Data Capture)",
            # "CNAF": "CNAF (CIMB Niaga Auto Finance)",
            # "KTA": "KTA (Kredit Tanpa Agunan)",
            # "ARO": "ARO (Automatic Roll Over)",
            # "PCL": "PCL (Permanent Credit Limit)",
            # "TCL": "TCL (Temporary Credit Limit)",
            # "POH": "POH (Pembantu Operasional Harian)",
            # "SPV": "SPV (Supervisor)",
            # "CC": "CC (Credit Card)",
            # "SLIK": "SLIK (Sistem Layanan Informasi Keuangan)",
            # "DSE": "DSE (Digital Sales Enablement)",
            },regex=True, inplace=True)
# print(df["Layer 1"].value_counts())

strip_all()
df.loc[df["Layer 3"].str.contains(": "), "Layer 3"] = df["Layer 3"].str.split(": ").str[1]
strip_all()
# print(df["Layer 1"].value_counts())
# print(df["Stage 1 - Layer 1"].value_counts())


In [7]:
stage+=1
add_stage(stage)

Stage 2: Split Layer 2 using Layer 3 prefixes

In [8]:
branchless_products = ["ATM/ CDM/ CRM",
                           "ATM",
                           "CDM",
                           "CRM",
                           "Kartu Debit",
                           "Debit Card",
                           "OCTO Clicks",
                           "OCTO Mobile",
                           "OCTO Pay",
                           "QRIS",
                           "Rekpon",
                           ]
split_layer3to2('Branchless', branchless_products)
# , "Branchless: ")

In [9]:
services_products = ["Biz Channel OCTO Merchant",
                     "Biz Channel",
                     "Social Media",
                     "Website",
                     "Contact Center",
                     "Whatsapp",
                     "Digital Lounge",
                     "Phone Banking",
                     "E-mail",
                     "EDC",
                     "Branch",
                     "Live Chat",
                     "Preferred",
                     "Chatbot",
                     "Transaksi 1500800",
                     "Vending Machine",
                     "Telesales",
                     "Restrukturisasi Kredit",
                     "Poin Xtra",
                     "Collection",
                     "Beasiswa",
                     "Asuransi",
                     ]
split_layer3to2('Services', services_products)
# , "Services: ")

In [10]:
lending_products = ["KTA/ PL",
                    "KPR",
                    "KPM",
                    "Refinancing (Multi Guna)",
                    "Xtra Kasbon",
                    ]
split_layer3to2('Lending', lending_products)
# , "Lending: ")

In [11]:
funding_products = ["Tabungan",
                    "Giro",
                    "Deposito",
                    "Obligasi",
                    "Reksa Dana",
                    ]
split_layer3to2('Funding', funding_products)
# , "Funding: ")

In [12]:
preferred_products = ["Poin Xtra",
                      "Customer Preferred Valas",
                      "Personal Credit Line (PCL)",
                      "PVI (Program Green Lane)",
                      ]
split_layer3to2('Program Preferred Blast', preferred_products)
#, "Preferred: ")

In [13]:
other_products = ["Poin Xtra",
                  "Asuransi",
                  "Vending Machine",
                  "Collection",
                  "Beasiswa",
                  "Telesales",
                  "Restrukturisasi Kredit",
                  ]
split_layer3to2('Other', other_products)

In [14]:
# split_layer3to2('Telesales Dok KTP Supplement', ["Telesales"])
# split_layer3to2('Done Telesales Dok KTP Supplement', ["Telesales Done"])
# split_layer3to2('Done Contact Center Dok KTP Supplement', ["Done Contact Center"])
# split_layer3to2('Contact Center Dok KTP Supplement', ["Contact Center"])
remove_postfix("Layer 3", "Contact Center")
remove_postfix("Layer 3", "Telesales")

In [15]:
stage+=1
add_stage(stage)

Stage 3: Credit Card

In [16]:
remove_prefix("Layer 3", "Credit Card")

In [17]:
credit_card_products = [
        "Credit Card",
        "Credit Card Aktivasi Accor Platinum",
        "Credit Card Aktivasi Accor World",
        "Credit Card Basic",
        "Credit Card General",
        "Credit Card Inactive & Dormant",
        "Credit Card Stickiness CPROT",
        "Credit Card Stickiness Non-Activated",
        "Credit Card Supplementary",
        "Credit Card Tendency Card",
    ]
split_layer2to3(credit_card_products, "Credit Card", "[", "] ")

In [18]:
stage+=1
add_stage(stage)

Stage 4: Feedback/Report and Report Status Checking -> Report Status

In [19]:
df.loc[df["Layer 1"] == "Report Status Checking", "Layer 3"] = "Status Open"
df.loc[df["Layer 1"] == "Feedback/ Report", "Layer 3"] = "Status Closed"
df.loc[df["Layer 1"] == "Report Status Checking", "Layer 1"] = "Report Status"
df.loc[df["Layer 1"] == "Feedback/ Report", "Layer 1"] = "Report Status"

In [20]:
stage+=1
add_stage(stage)

Stage 5: Grouping Labels

In [21]:
# 
# 
# 
# Grouping Layer 1, Typo
group_label("Layer 1", ["Spam", "Ongoing", "Miscellaneous", "Test"], "Other")
group_label("Layer 1", ["Sales", "Campaign"], "Campaign/ Sales")

In [22]:
# 
# 
# 
# Grouping Layer 3, Typo
group_label("Layer 3", ["Promo/ Voucher", "Promo/ Voucher/milliage", "Promo/ Voucher/ Milliage", "Promo/ Voucher/Milliage"], "Promo/ Voucher/ Mileage")
group_label("Layer 3", ["Prank Call/ Spam Call/ Bogus Call"], "Spam")
group_label("Layer 3", ["Penerbitan", "Penggantian"], "Penerbitan/ Penggantian")

In [23]:
stage+=1
add_stage(stage)

Stage 6: Swapping Layers

In [24]:
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Salah Nama",layer_name_2_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Test",layer_name_2_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Chat lanjutan",layer_name_2_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Info belum lengkap",layer_name_2_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_2_contain="OCTO ")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Whatsapp")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Blast")


In [25]:
stage+=1
add_stage(stage)

Stage 7: Targeted Cleaning

In [26]:
# remove_prefix("Layer 3", "Sales")
# remove_prefix("Layer 3", "XKB")

# remove_prefix("Layer 2", "Done Contact Center")
# remove_prefix("Layer 2", "Done Telesales")

# PCL = Personal Credit Line

In [27]:
df.loc[(df["Layer 1"] == "Other") & (df["Layer 2"] == "Salah layanan"), "Layer 2"] = "Other"
df.loc[(df["Layer 1"] == "Other") & (df["Layer 2"] == "Spam"), "Layer 2"] = "Other"    
df.loc[df["Layer 3"] == "No", "Layer 3"] = "Tidak Setuju"
df.loc[df["Layer 2"] == "Contact", "Layer 2"] = "Contact Center"

In [28]:
stage+=1
add_stage(stage)

Stage 8: Cleaning Empty Data

In [29]:
strip_all()
df.loc[df["Layer 1"] == "", "Layer 1"] = None
df.loc[df["Layer 2"] == "", "Layer 2"] = None
df.loc[df["Layer 3"] == "", "Layer 3"] = None
df.fillna("Other", inplace=True)
df.drop_duplicates(inplace=True)
# df.sort_values(by=["Layer 1","Layer 2","Layer 3"],inplace=True)
# print(df.size//3)

In [30]:
stage+=1
add_stage(stage)

In [31]:
# print(df["Layer 1"].unique().size)
# print(df["Layer 2"].unique().size)
# print(df["Layer 3"].unique().size)
# print(df["Layer 1"].value_counts())
# print(df["Layer 2"].value_counts())
# print(df["Layer 3"].value_counts())

# print("Layer 1 - Layer 2")
# print(df[["Layer 1", "Layer 2"]].value_counts().size)
# print(df[["Layer 1", "Layer 2"]].value_counts())

# print("Layer 2 - Layer 3")
# print(df[["Layer 2", "Layer 3"]].value_counts().size)
# print(df[["Layer 2", "Layer 3"]].value_counts())

# print("Layer 1 - Layer 3")
# print(df[["Layer 1", "Layer 3"]].value_counts().size)
# print(df[["Layer 1", "Layer 3"]].value_counts())

Display

In [32]:
df[["Layer 1", "Layer 2", "Layer 3"]].value_counts().size

1145

In [33]:
# df.to_csv("C:\\Users\\CX16230X\\Documents\\Projects\\ATHENA\\disposition-code\\disp.csv")

In [34]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# df
print(df[["Layer 1", "Layer 2", "Layer 3"]])

              Layer 1                            Layer 2  \
0            Complain                                ATM   
1            Complain                                ATM   
2            Complain                                ATM   
3            Complain                                ATM   
4            Complain                                ATM   
5            Complain                                ATM   
6            Complain                                ATM   
7            Complain                                ATM   
8            Complain                                ATM   
9            Complain                                ATM   
10           Complain                                CDM   
11           Complain                                CDM   
12           Complain                                CDM   
13           Complain                                CDM   
14           Complain                                CDM   
15           Complain                   